#### `Data information:`

- `x` genomes (strains) 
- `x` different treatments (growth conditions)
- `x3` replicates
- plus `x` negative controls (referred to as "blanks")

In [ ]:
import pandas as pd
import os

`Prepare Matrix for PCA`
- Cluster the different strains (groupby genome ID) and calculate the sum (0.0 no feature detected in all 3 treatments in all 3 replicates - 9.0 feature detected in all 3 treatments in all 3 replicates)

In [ ]:
path= os.path.join("pyOpenMS_old_results", "data_analysis", "tables_big_data")
isExist= os.path.exists(path)
if not isExist:
    os.mkdir(path)

Here you can group according to what you would like to compare. E.g. genomes or treatments. 

In [ ]:
Matrix= pd.read_csv(os.path.join(path, "Matrix_Clean.csv"), sep="\t")
Matrix= Matrix.set_index(["mz", "RT"])
Matrix= Matrix.fillna(0)
Matrix= Matrix.where(Matrix==0, 1)
Matrix= Matrix.sort_index(axis=1)
cols= Matrix.columns
Matrix= Matrix.transpose()
Matrix= Matrix.reset_index()
Matrix['genomeID']=Matrix['index'].str.extract(r'(NBC_?\d*)')
Matrix['genomeID_MDNA']=Matrix['index'].str.extract(r'(MDNAWGS?\d*|MDNA_WGS_?\d*)')
Matrix['genomeID']=Matrix['genomeID'].fillna(Matrix['genomeID_MDNA'])
Matrix['treatments']= Matrix['index'].str.extract(r'(ISP2|DNPM|FPY12)')
Matrix= Matrix.drop(columns=["genomeID_MDNA"])
Matrix=Matrix.set_index(["index"])
Grouped= Matrix.groupby("treatments").sum() #or genome ID
Grouped= Grouped.transpose()
Grouped= Grouped.reset_index()
Grouped.to_csv(os.path.join(path, "Matrix_Grouped.csv"), sep="\t", index=None)
Grouped= Grouped.set_index(["mz", "RT"])
Grouped

In [ ]:
FeatureMatrix= pd.read_csv(os.path.join(path, "Matrix_Grouped.csv"), sep='\t')
FeatureMatrix

#### `2) PCA and outlier analysis`

##### `(i) Data pre-treatment`

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as shc 

from collections import Counter
#from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
#from sklearn.model_delection import ParameterGrid
from sklearn.datasets import load_iris
from sklearn import metrics

In [ ]:
Matrix= pd.read_csv(os.path.join(path,"Matrix_Grouped.csv"), sep="\t")
Matrix= Matrix.set_index(["mz", "RT"])
Matrix= Matrix.transpose()
Matrix

- `Centering`
    - Import the grouped csv files and perform mean centering to focus on the fluctuating part of the data  

In [ ]:
Matrix_centered = Matrix.apply(lambda x: x-x.mean())
Matrix_centered

- Explore if you have outliers with unbiasted skew

In [ ]:
Matrix_centered.skew(axis=0, skipna = True)

In [ ]:
Matrix_centered.describe()

- `Standardization`
    - This will make sure that all the features are centred around the mean value with a standard deviation value of 1. This is the best to use if your feature is normally distributed.

In [ ]:
import math
def variance(data, ddof=0):
    n = len(data)
    mean = sum(data) / n
    return sum((x - mean) ** 2 for x in data) / (n - ddof)


def stdev(data):
    var = variance(data)
    std_dev = math.sqrt(var)
    return std_dev
    
Matrix_standardized= Matrix_centered.apply(lambda x: (x-x.mean()) / stdev(x))
Matrix_standardized

In [ ]:
x= Matrix_centered
scaled_data = Matrix_standardized
f, ax = plt.subplots(1,2)
sns.distplot(x, ax=ax[0], color='y')
ax[0].set_title("Original data")
sns.distplot(scaled_data, ax=ax[1], color='g')
ax[1].set_title("Scaled data")
plt.show()

In [ ]:
# So, as you know, the principle components of the data are the
# dimensions along which the data varies the most. The data
# here is 4 dimensional (since there are 4 features) and thus
# there are 4 total principle components.

# The first principle component will be the line along which the
# data varies the most. The second will be the line along with the
# data varies the second monst, and so on. The sum of the variances
# of all the principle components will be the entire variance of the
# dataset.

# Okay, so let's use the PCA function to get all 4 principle components.

# As always with the sklearn package, we first have to create and save
# a function "object":
pca = PCA(n_components=3)

# We can actually use this function object to get the first 4 principle
# components of any data. Let's do it for our data:
data_pca = pca.fit_transform(Matrix_standardized)

# Now that we've called fit_transform, the pca object some attributes
# that includes the data points transformed along the principle components.
# So let's plot the data points along the first two principle components.
# We'll use the matplotlib.pyplot package to do this:
plt.scatter(data_pca[:, 0], data_pca[:, 1])
plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.title("Our data along the first two PCs")
plt.grid(True)
plt.show()

# The explained variance is also included in the pca object.
# So we can plot that as well:
plt.plot(pca.explained_variance_ratio_,
        label="Percent of variance explained for each PC")
plt.plot([sum(pca.explained_variance_ratio_[:i]) for i in range(1, 5)],
        label="Cumulative percent of variance explained")
plt.legend()
plt.grid(True)
plt.xticks(range(4), ["%0.0f" % i for i in range(1, 5)])
plt.ylabel("%")
plt.xlabel("PC")
plt.title("Variance explained by each PC")
plt.show()

#make a plot where you zoom in
#volcano plots
# So now we see empirical evidence that the sum of all 4 variances
# equals the total variance in the data set (shown by the orange line) :)

Interactive PCA plots inspired from https://www.kaggle.com/maniyar2jaimin/interactive-plotly-guide-to-pca-lda-t-sne

In [ ]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.subplots as tls

import seaborn as sns
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

# Import the 3 dimensionality reduction methods
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
# Standardize the data
from sklearn.preprocessing import StandardScaler
X = Matrix_standardized.values
X_std = StandardScaler().fit_transform(X)
# Calculating Eigenvectors and eigenvalues of Cov matirx
mean_vec = np.mean(X_std, axis=0)
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eigh(cov_mat) #ASK
# Create a list of (eigenvalue, eigenvector) tuples
eig_pairs = [ (np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]
# Sort the eigenvalue, eigenvector pair from high to low
eig_pairs.sort(key = lambda x: x[0], reverse= True)

# Calculation of Explained Variance from the eigenvalues
tot = sum(eig_vals)
var_exp = [(i/tot)*100 for i in sorted(eig_vals, reverse=True)] # Individual explained variance
cum_var_exp = np.cumsum(var_exp) # Cumulative explained variance

In [ ]:
trace1 = go.Scatter(
    x=list(range(10)),
    y= cum_var_exp,
    mode='lines+markers',
    hoverinfo= "all",
    name="'Cumulative Explained Variance'",
    line=dict(
        shape='spline',
        color = 'goldenrod'
    )
)
trace2 = go.Scatter(
    x=list(range(10)),
    y= var_exp,
    hoverinfo= "all",
    mode='lines+markers',
    name="'Individual Explained Variance'",
    line=dict(
        shape='linear',
        color = 'black'
    )
)
fig = tls.make_subplots(insets=[{'cell': (1,1), 'l': 0.7, 'b': 0.3}],
                          print_grid=True)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2,1,1)
fig.layout.xaxis = dict(range=[0, 9.5])
fig.layout.yaxis = dict(range=[0, 110])

py.iplot(fig, filename='inset example')

In [ ]:
X= Matrix_standardized.values

# Standardising the values
X_std = StandardScaler().fit_transform(X)

# Call the PCA method with 50 components. 
pca = PCA(n_components=2)
pca.fit(X_std)
X_5d = pca.transform(X_std)

In [ ]:
trace0 = go.Scatter(
    x = X_5d[:,0],
    y = X_5d[:,1],
    name = "Target",
    hoverinfo = 'all',
    mode = 'markers',
    showlegend = False,
    marker = dict(
        size = 8,
        colorscale ='Jet',
        showscale = False,
        line = dict(
            width = 2,
            color = 'rgb(255, 255, 255)'
        ),
        opacity = 0.8
    )
)
data = [trace0]

layout = dict(title = 'PCA (Principal Component Analysis)',
              hovermode= 'closest',
              yaxis = dict(zeroline = False),
              xaxis = dict(zeroline = False),
              showlegend= True
             )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')

In [ ]:
Q1=Matrix_standardized.quantile(0.25)
Q3=Matrix_standardized.quantile(0.75)
IQR=Q3-Q1
upper_val= Q3+1.5*IQR #most frequently occuring features    
lower_val= Q1-1.5*IQR #most rarely occuring features

Only investigate the rarely occuring metabolites(lower_val):

In [ ]:
lower_val.to_csv(os.path.join(path, "lower_val.csv"), sep="\t")
lower_val

In [ ]:
Matrix_standardized= Matrix_standardized.transpose()
Matrix_standardized= Matrix_standardized.reset_index()
Matrix_standardized

In [ ]:
lower_val= pd.read_csv(os.path.join(path, "lower_val.csv"), sep="\t")
lower_val= lower_val.rename(columns={"0":"rarest_features"})
Matrix_scaled= pd.merge(Matrix_standardized, lower_val, on=["mz", "RT"])
Matrix_scaled= Matrix_scaled.set_index(["mz", "RT"])
Matrix_scaled= Matrix_scaled.round(decimals=5)
Matrix_scaled

In [ ]:
Matrix_standardized

In [ ]:
Matrix_standardized=Matrix_standardized.set_index(["mz", "RT"])
cols= Matrix_standardized.columns
outliers=Matrix_scaled

for col in cols:
    for idx in outliers.index:
        if outliers["Lower_limit"][idx]<= outliers[col][idx]:
            outliers.loc[idx, col]=np.nan

outliers

In [ ]:
outliers= outliers.drop(columns=["Upper_limit", "Lower_limit"])
outliers= outliers.dropna(how="all")
outliers= outliers.transpose()
outliers= outliers.dropna(how="all")
outliers= outliers.transpose()
outliers.to_csv(os.path.join(path, "outliers.csv"), sep="\t")
outliers

`Outlier GNPS annotation`

In [ ]:
df= pd.read_csv(os.path.join("resources", "MS2_LIBRARYSEARCH_all_identifications.tsv"), sep='\t', encoding='latin-1')
df.drop(df.index[df['IonMode'] == "negative"], inplace=True)
df.drop(df.index[df['MZErrorPPM'] > 20.0], inplace=True)
GNPS=df.drop(columns=["PI", "Adduct", "IonMode", "Organism", "MZErrorPPM", "SpecMZ"])
GNPS=GNPS.rename(columns= {"RT_Query": "RetentionTime"})
GNPS=GNPS.drop_duplicates(subset="Compound_Name", keep='first')
GNPS

In [ ]:
outliers.insert(0, 'GNPS_IDs', '')

for i, mz, rt in zip(outliers.index, outliers['mz'], outliers['RT']):
    hits = []
    for name, GNPS_mz, GNPS_rt, in zip(GNPS['Compound_Name'], GNPS['Precursor_MZ'], GNPS['RetentionTime']):
        mass_delta = (abs(GNPS_mz-mz)/GNPS_mz)*1000000.0 if GNPS_mz != 0 else np.nan
        if (GNPS_rt >= rt-30.0) & (GNPS_rt <= rt+30.0) & (mass_delta<= 20.0):
            hit = f'{name}'
            if hit not in hits:
                hits.append(hit)
    outliers['GNPS_IDs'][i] = ' ## '.join(hits)

outliers.to_csv(os.path.join("results", "annotations", "GNPS_annotated_outliers.tsv"), sep='\t', index = False)
outliers

In [ ]:
outliers= outliers[outliers.GNPS_IDs == '']
outliers= outliers.drop(columns= "GNPS_IDs")
outliers= outliers.set_index(["mz", "RT"])
outliers_tocsv= outliers.reset_index()
outliers_tocsv.to_csv(os.path.join("results", "annotations", "outliers_unknowns.tsv"), sep="\t", index =None)
outliers